# Running ASMC with ground-truth data

Below we simulate and evaluate the ability of ASMC to decode TMRCA from a ground-truth simulation using `msprime`.

Note: for this notebook you must also insall `asmc-preparedecoding`, `tszip`, `msprime`, `bcftools`, and `tabix`:

```bash
pip install asmc-preparedecoding tszip msprime
sudo apt install bcftools tabix
```

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import msprime as msp
import tszip
import pathlib

from asmc.asmc import *
from asmc.preparedecoding import *

import os
import sys

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

%matplotlib inline

print("numpy version: {}".format(np.__version__)) 
print("msprime version: {}".format(msp.__version__)) 
print("pandas version: {}".format(pd.__version__)) 
print("tszip version: {}\n".format(tszip.__version__))

# Utility / Plotting Functions 

Here we've defined the following functions:
* `sim_pop_demography`: a function to construct a Demography object in `msprime` from a pre-existing ASMC demography file.
* `get_true_tmrca`: obtain the true tmrca per-locus between two haplotype IDs at a specified set of positions.
* `tmrca_rmse`: RMSE between two vectors of per-locus TMRCA estimates.
* `plot_asmc_single_pair`: plot pairwise TMRCA as a step-wise plot and the ratio of per-locus TMRCA estimates in two subplots.

In [ ]:
def sim_pop_demography(demo_file):
    """ Simulate a single population. 
        input:
            demo_file - two column file ([Generations,NE]) 
                NOTE: assumed to be haploid
        output: 
            demography: msp.Demography object
    """
    df = pd.read_table(demo_file, header=None)
    df.columns  = ['GEN', 'NE']
    demography = msp.Demography()
    demography.add_population(name="A", initial_size=1e4)
    for t,n in zip(df.GEN.values, df.NE.values):
        # NOTE: we have to divide because msprime expects a diploid demography for working with VCFs
        demography.add_population_parameters_change(t, initial_size=n/2, population="A")
    return(demography)

In [ ]:
def get_true_tmrca(ts, id1, id2, snp_locations=None):
    """ Obtain the true tmrcas between id1 and id2 samples 
        Arguments:
         - ts : tskit Tree-Sequence object
         - id1: int specifying haplotype 1
         - id2: int specifying haplotype 2
         - snp_locations: np.array of snp_locations to test against (default: None)
    """
    if snp_locations is None:
        snp_locations = np.array([s.position for s in ts.sites()])
        assert snp_locations.size > 0
    test_windows = [0.]
    for i in snp_locations:
        test_windows.append(i-0.1)
        test_windows.append(i)
    test_windows.append(ts.sequence_length)
    true_tmrcas = ts.diversity([id1,id2], 
                               mode='branch',
                               windows=test_windows,
                               span_normalise=True)[1::2] / 2
    assert snp_locations.size == true_tmrcas.size
    return(true_tmrcas, snp_locations)

def tmrca_rmse(ts1,ts2):
    """ RMSE between two estimates of tmrca (ts1, ts2)."""
    assert ts1.size == ts2.size
    assert ts1.ndim == ts2.ndim
    assert ts1.ndim == 1
    rmse = np.sqrt(np.nanmean((ts1 - ts2)**2))
    return(rmse)

In [ ]:
def plot_yx(ax, **kwargs):
    """Plot the y=x line in a matplotlib axis."""
    lims = [
        np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
        np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
    ]
    # now plot both limits against each other
    ax.plot(lims, lims, **kwargs)


def debox(ax):
    """Remove the top and right spines of a plot."""
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    
def plot_asmc_single_pair(fig, phys_bp, true_tmrcas, est_tmrcas, labels):
    """ Generate a plot of the undrelying tmrcas and the estimated tmrcas
        NOTE: also generates a scatter plot of true vs estimated tmrcas
        
        Arguments:
        ----------
        - fig: matplotlib.figure for generating the figure.
        - phys_bp: numpy array of positions.
        - true_tmrca: numpy array of true tmrca values.
        - est_tmrcas: list of estimated tmrcas (list of numpy arrays).
        - labels: list of labels for each tract of posterior tmrcas
    """
    assert phys_bp.size == true_tmrca.size
    for x in est_tmrcas:
        assert phys_bp.size == x.size
    assert len(labels) == len(est_tmrcas)
    assert len(est_tmrcas) > 0
    gs = fig.add_gridspec(1, 3)
    ax0 = fig.add_subplot(gs[0, :2])
    ax1 = fig.add_subplot(gs[0, 2])
    # Plot a version of this here ... 
    ax0.step(phys_bp, true_tmrca, label=r'True TMRCA')
    for lbl, est_t in zip(labels, est_tmrcas):
        ax0.step(phys_bp, est_t, label=lbl)
    ax0.set_yscale('log')
    ax0.set_ylabel(r'$T_{MRCA}$')
    ax0.set_xlabel('Position (bp)')
    
    ax0.legend(fontsize=10)
    ax1.scatter(true_tmrca, est_tmrcas[0], 
                s=10, alpha=0.1, 
                label=r'$\frac{T}{\hat{T}}=%0.2f$' % np.nanmedian(true_tmrca/est_tmrcas[0]))
    ax1.set_aspect('equal', adjustable='box')
    ax1.legend()
    ax1.set_xlabel(r'True $T_{MRCA}$')
    ax1.set_ylabel(r'Posterior Mean $T_{MRCA}$')
    plot_yx(ax1, color='r', linestyle='--')
    debox(ax1); debox(ax0);
    return(fig, [ax0,ax1])

# Simulating Ground-Truth Data

Our goal here is to simulate ground-truth data in `msprime` based on the established ASMC demography. 

In [ ]:
## Parameter Settings

data_dir = pathlib.Path('.').resolve().parent / 'ASMC_data'

outfix = 'test' # this is the outfix for all of our datasets that are output
demo_file = str(data_dir / 'demographies' / 'CEU.demo') # Simulate under the CEU demography ...  
demography = sim_pop_demography(demo_file) 

# Simulate 300 diploid samples (600 haploid) 
ts = msp.sim_ancestry(samples={"A": 300}, 
                      demography=demography, 
                      sequence_length=10e6, 
                      recombination_rate=1e-8, 
                      ploidy=2, # msprime is diploid by default, but we make it explicit here 
                      random_seed=42)

mts = msp.sim_mutations(ts, 
                        rate=1e-8, 
                        random_seed=42, 
                        model=msp.BinaryMutationModel())

# Save the tree-sequence as a zipped file for now 
tszip.compress(mts, f"{outfix}.ts.tsz")

The following shell commands leverage the popular `bcftools` utility to convert to a `haps.gz` and `map.gz` fileset for inference under ASMC:

In [ ]:
# Convert our zipped tree-sequence to a haps.gz file with our outfix  
os.system("tszip -d -k --force {}.ts.tsz".format(outfix))
os.system("tskit vcf {}.ts | bcftools view -v snps -m2 -M2 | \
    bcftools +fill-tags | bgzip -@8 > {}.vcf.gz".format(outfix, outfix))
os.system("echo -e \"CHR\tSNP\tA1\tA2\tMAF\tNCHROBS\" > {}.seq.frq; \
    bcftools query -f \"%CHROM\t%ID\t%REF\t%ALT\t%MAF\t%AN\n\" {}.vcf.gz >> {}.seq.frq".format(outfix,outfix, outfix))
os.system("tabix -f {}.vcf.gz".format(outfix))
os.system("bcftools query -f \'%CHROM\t%ID\t%POS\n\' \
    {}.vcf.gz  | awk \'{{OFS=\"\t\"; print $1,$2,(2E-8 * $3 *100),$3}}\' | bgzip -@4 > {}.seq.map.gz".format(outfix, outfix))
os.system("bcftools convert --hapsample {}.seq.hap.gz,{}.seq.samples {}.vcf.gz".format(outfix, outfix, outfix));

The next block indicates how we can construct and save decoding quantities that are specific to our simulation:

In [ ]:
## Generating the decoding quantities just for this simulation (based on the sequencing data)
## NOTE: this step can take 1-2 minutes
dq = prepare_decoding(
    demography=demo_file,
    discretization=[[30.0, 15], [100.0, 15], 39], # 15 blocks of 30 generations, 15 blocks of 100 generations, 39 quantiles...
    frequencies=f"{outfix}.seq.frq",
    samples=300,
    mutation_rate=1e-8)
dq.save_decoding_quantities(f"{outfix}")

In [ ]:
id1, id2 = 0,1

asmc_seq = ASMC(f"{outfix}.seq", f"{outfix}.decodingQuantities.gz", decoding_mode="sequence")
asmc_seq.set_store_per_pair_posterior_mean(True)
asmc_seq.set_store_per_pair_map(True)
asmc_seq.decode_pairs([id1],[id2]) # We'll just decode for a single pair 
return_seq = asmc_seq.get_copy_of_results()

# Plotting Results... 
fig = plt.figure(constrained_layout=True, figsize=(8,4))
true_tmrca, phys_bp = get_true_tmrca(mts, id1=id1, id2=id2)
posterior_mean_tmrca_seq = return_seq.per_pair_posterior_means[0]
_, [ax0, ax1] = plot_asmc_single_pair(fig, phys_bp, true_tmrca, 
                                      [posterior_mean_tmrca_seq], 
                                      ['Posterior Mean (Sequence-mode)'])
# Only show a limited view of the posterior decoding to avoid messiness ...
ax0.set_xlim(2e6, 2.5e6);
ax1.set_yscale('log'); ax1.set_xscale('log');

## Revisiting Posterior Decoding using Array Data

The above example primarily focuses on leveraging the "sequencing" mode of ASMC, but the more recognizeable use-case is when one has genotyping array data. To simulate the effects of genotyping arrays, we will use an approximation where we build decoding quantities from all variants where the minor allele frequency is $> 1\%$ (using the `-q 0.01:minor` flag in `bcftools`).

In [ ]:
# Convert our zipped tree-sequence to a haps.gz file with our outfix  
os.system("tszip -d -k --force {}.ts.tsz".format(outfix))
os.system("tskit vcf {}.ts | bcftools view -v snps -m2 -M2 -q 0.01:minor | \
    bcftools +fill-tags | bgzip -@4 > {}.vcf.gz".format(outfix, outfix))
os.system("echo -e \"CHR\tSNP\tA1\tA2\tMAF\tNCHROBS\" > {}.array.frq; \
    bcftools query -f \"%CHROM\t%ID\t%REF\t%ALT\t%MAF\t%AN\n\" {}.vcf.gz >> {}.array.frq".format(outfix,outfix, outfix))
os.system("tabix -f {}.vcf.gz".format(outfix))
os.system("bcftools query -f \'%CHROM\t%ID\t%POS\n\' \
    {}.vcf.gz  | awk \'{{OFS=\"\t\"; print $1,$2,(2E-8 * $3 *100),$3}}\' | bgzip -@4 > {}.array.map.gz".format(outfix, outfix))
os.system("bcftools convert --hapsample {}.array.hap.gz,{}.array.samples {}.vcf.gz".format(outfix, outfix, outfix));

dq = prepare_decoding(
    demography=demo_file,
    discretization=[[30.0, 15], [100.0, 15], 39], # 15 blocks of 30 generations, 15 blocks of 100 generations, 39 quantiles...
    frequencies=f"{outfix}.array.frq",
    samples=300,
    mutation_rate=1e-8)
dq.save_decoding_quantities(f"{outfix}.array")

In [ ]:
# We need to import pandas to read the positions
import pandas as pd
id1, id2 = 0,1

asmc_array = ASMC(f"{outfix}.array", f"{outfix}.array.decodingQuantities.gz", decoding_mode="array")
asmc_array.set_store_per_pair_posterior_mean(True)
asmc_array.set_store_per_pair_map(True)
asmc_array.decode_pairs([id1],[id2])
return_array = asmc_array.get_copy_of_results()

# Plotting Results...
df = pd.read_table(f"{outfix}.array.map.gz", header=None)
fig = plt.figure(constrained_layout=True, figsize=(8,4))

true_tmrca, phys_bp = get_true_tmrca(mts, id1=id1, id2=id2, snp_locations=df[3].values)
posterior_mean_tmrca_array = return_array.per_pair_posterior_means[0]
_, [ax0, ax1] = plot_asmc_single_pair(fig, phys_bp, true_tmrca, 
                                      [posterior_mean_tmrca_array], 
                                      ['Posterior Mean (Array-mode)'])

ax0.set_xlim(2e6, 2.5e6);
ax1.set_yscale('log'); ax1.set_xscale('log');

## Conclusion

In this tutorial we've explored how to:

1. Setup a simulation & visualization scheme to test ASMC applied to ground-truth data
2. Highlighted the ability to generate decoding quantities directly based on the data
3. Explored the differences on running between true sequence data or simulated array data


We hope that this provides some key insights into the useability of `ASMC` within a python environment. If you have any questions on this tutorial, or interested in different features of ASMC, please feel free to submit an issue. 